In [1]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
from sklearn.utils import class_weight
import tensorflow.keras.layers as layers
from data_generator import NpyDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from tqdm import tqdm
from collections import Counter
import numpy as np
from tensorflow.keras.optimizers import RMSprop, SGD, Adam,Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.layers import *
import einops

2024-08-11 16:22:50.501267: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/kashraf/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_path = "/media/kashraf/TOSHIBA EXT/Dissertation/stage2/EEG movie/Theta/train"
test_path = "/media/kashraf/TOSHIBA EXT/Dissertation/stage2/EEG movie/Theta/test"
train_gen = NpyDataGenerator(train_path,batch_size=8)
validation_gen = NpyDataGenerator(test_path,batch_size=1,shuffle=False)
print("Training samples: ",train_gen.num_samples)
print("Test samples: ",validation_gen.num_samples)

Training samples:  4200
Test samples:  1800


In [3]:
class Conv2Plus1D(keras.layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        super().__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.seq = keras.Sequential([
            # Spatial decomposition
            layers.Conv3D(filters=filters,
                          kernel_size=(kernel_size[0], kernel_size[1], 1),
                          padding=padding),
            # Temporal decomposition
            layers.Conv3D(filters=filters,
                          kernel_size=(1, 1, kernel_size[2]),
                          padding=padding)
        ])

    def get_config(self):
        config = super().get_config()
        config.update({
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'padding': self.padding
        })
        return config

    @tf.function
    def call(self, x):
        return self.seq(x)



class ResidualMain(keras.layers.Layer):
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.seq = keras.Sequential([
            Conv2Plus1D(filters=filters,
                        kernel_size=kernel_size,
                        padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters=filters, 
                        kernel_size=kernel_size,
                        padding='same'),
            layers.LayerNormalization()])

    def get_config(self):
        config = super().get_config()
        config.update({
            'filters': self.filters,
            'kernel_size': self.kernel_size
        })
        return config

    @tf.function
    def call(self, x):
        return self.seq(x)

    
class Project(keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
        self.seq = keras.Sequential([
            layers.Dense(units),
            layers.LayerNormalization()
        ])
    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
        })
        return config
    @tf.function
    def call(self, x):
        return self.seq(x)
    
def add_residual_block(input, filters, kernel_size):
    out = ResidualMain(filters, kernel_size)(input)
    res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)

    return layers.add([res, out])

class ResizeVideo(keras.layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.experimental.preprocessing.Resizing(self.height, self.width)
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'height': self.height,
            'width': self.width,
        })
        return config
    @tf.function
    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c',t = old_shape['t'])
        return videos


## Final Model

In [4]:
HEIGHT = 64
WIDTH = 64
TIME = 176
CHANNEL = 1
input_shape = (None,HEIGHT, WIDTH,TIME, CHANNEL)
input = layers.Input(shape=(input_shape[1:]))
x = input

x = Conv2Plus1D(filters=32, kernel_size=(5, 5,5), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = MaxPool3D(pool_size=(2, 2, 2))(x)
x = Dropout(0.3)(x)

# Block 1
x = add_residual_block(x, 32, (3, 3, 3))
x = MaxPool3D(pool_size=(2, 2, 2))(x)

# Block 2
x = Conv2Plus1D(filters=32, kernel_size=(3, 3,3), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = MaxPool3D(pool_size=(2, 2, 2))(x)
x = Dropout(0.3)(x)

# Block 3
x = add_residual_block(x, 64, (3, 3, 3))
x = MaxPool3D(pool_size=(2, 2, 2))(x)


# Block 4
x = Conv2Plus1D(filters=32, kernel_size=(3, 3,3), padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
# x = ResizeVideo(HEIGHT // 2, WIDTH // 2)(x)
x = MaxPool3D(pool_size=(2, 2, 1))(x)

# x = layers.GlobalAveragePooling3D()(x)
x = layers.GlobalAveragePooling3D()(x)
# x = Dense(units=256, activation="relu")(x)
# x = Dropout(0.3)(x)
x = Dense(units=512, activation="relu")(x)
x = Dropout(0.3)(x)

x = layers.Dense(units=4, activation='softmax')(x)

model = keras.Model(input, x)

2024-08-11 16:22:52.357196: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-08-11 16:22:52.358315: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-08-11 16:22:52.389441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-08-11 16:22:52.389607: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2024-08-11 16:22:52.389619: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-08-11 16:22:52.390758: I tensorflow/stream_executor/platfor

In [5]:
model.summary()
# tf.keras.utils.plot_model(model, expand_nested=True, dpi=60, show_shapes=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 176, 0                                            
__________________________________________________________________________________________________
conv2_plus1d (Conv2Plus1D)      (None, 64, 64, 176,  5984        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 176,  128         conv2_plus1d[0][0]               
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 64, 64, 176,  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
             
checkpoint = ModelCheckpoint("/media/kashraf/TOSHIBA EXT/Dissertation/stage2/modeling/weights/3D_CNN_EEG_movie.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 20,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.01,
                              patience = 5,
                              verbose = 1,
                              min_delta = 0.0001)

# we put our call backs into a callback list
callbacks = [ checkpoint,reduce_lr,earlystop]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])


epochs = 100

history= model.fit(   
    train_gen,
    epochs = epochs,
    callbacks =callbacks,
    validation_data = validation_gen)

Epoch 1/100


2024-08-11 16:22:53.562913: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-08-11 16:22:53.579491: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3699850000 Hz
2024-08-11 16:22:54.591859: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-08-11 16:22:54.766339: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


525/525 [==============================] - 180s 337ms/step - loss: 1.6008 - accuracy: 0.2454 - val_loss: 1.5269 - val_accuracy: 0.2500

Epoch 00001: val_loss improved from inf to 1.52694, saving model to /media/kashraf/TOSHIBA EXT/Dissertation/stage2/modeling/weights/3D_CNN_EEG_movie.h5
Epoch 2/100
525/525 [==============================] - 177s 337ms/step - loss: 1.5324 - accuracy: 0.2689 - val_loss: 1.7162 - val_accuracy: 0.2500

Epoch 00002: val_loss did not improve from 1.52694
Epoch 3/100
525/525 [==============================] - 180s 343ms/step - loss: 1.4899 - accuracy: 0.2853 - val_loss: 1.5760 - val_accuracy: 0.2533

Epoch 00003: val_loss did not improve from 1.52694
Epoch 4/100
525/525 [==============================] - 188s 357ms/step - loss: 1.4604 - accuracy: 0.3032 - val_loss: 1.5867 - val_accuracy: 0.2533

Epoch 00004: val_loss did not improve from 1.52694
Epoch 5/100
525/525 [==============================] - 188s 357ms/step - loss: 1.4241 - accuracy: 0.3161 - val_loss